## Utils

In [2]:
import telnetlib 
import json

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(hsh):
    request = json.dumps(hsh).encode()
    tn.write(request)

HOST = 'socket.cryptohack.org' 

/var/folders/s6/v17rrjkx205gt68br6qh7zqr0000gn/T/ipykernel_1504/3786777262.py:1: DeprecationWarning: 'telnetlib' is deprecated and slated for removal in Python 3.13
  import telnetlib


# ONE TIME PAD

### Gotta Go Fast 

Since server uses current time as input to SHA256 as key we can send 2 challenges fast and get same key for encryption of flag and our message

In [13]:
from pwn import xor 
PORT = 13372
tn = telnetlib.Telnet(HOST, PORT)

readline()

m_prime = b'crypto{????????????????????}'
forgery = {}
forgery['option'] = 'encrypt_data'
forgery['input_data'] = m_prime.hex() 

json_send(forgery)
r = json_recv()
c_prime = bytes.fromhex(r['encrypted_data']) 
k= xor(c_prime, m_prime)

json_send({'option':'get_flag'})
r = json_recv()
c = bytes.fromhex(r['encrypted_flag'])
print(xor(c, k).decode())

crypto{t00_f4st_t00_furi0u5}


### No leaks

So it checks if ciphertext has same byte as flag, so basically if we get ciphertext back, we can remove from guesses all the characters we've seen there for our bytes. 

In [36]:
import base64 

PORT = 13370
tn = telnetlib.Telnet(HOST, PORT)

readline()
FLAG = "crypto{????????????}"

guesses = [] 
for _ in range(len(FLAG)-8): 
    guesses.append([i for i in range(33,127)])

while True: 
    progress = sum([len(i) for i in guesses]) 
    
    if progress == 12: 
        break 

    json_send({'msg': 'request'})
    r = json_recv()
    
    if "error" not in r:  
        c = base64.b64decode(r['ciphertext'])
        c = c[7:len(FLAG)-1]
        for i in range(len(c)): 
            try: 
                guesses[i].remove(c[i]) 
            except Exception: 
                continue 
f = "crypto{"           
for i, g in enumerate(guesses): 
    f+=chr(g[0]) 
print(f+"}")

crypto{unr4nd0m_07p}


# SECRET SHARING SCHEMES

### Armory

https://en.wikipedia.org/wiki/Shamir%27s_secret_sharing

We've got the evaluation of polynomial $y$ and $a_1$ given to us, since polynomial is of degree 2 and following the code we derive that $x=a1$ 
and that $a_2=H(a_1)$ $H$ - SHA256. 

$y = a_2x^2 + a_1x + f \bmod p$
 
$f = y - a_2a_1^2 - a_1^2 \bmod p$ 


In [9]:
from Crypto.Util.number import long_to_bytes, bytes_to_long
import hashlib  
a1,y = (105622578433921694608307153620094961853014843078655463551374559727541051964080, 
 25953768581962402292961757951905849014581503184926092726593265745485300657424)

p = 77793805322526801978326005188088213205424384389488111175220421173086192558047
a2 = bytes_to_long(hashlib.sha256(long_to_bytes(a1)).digest()) 

print(long_to_bytes((y-a2*a1*a1 - a1*a1)%p).decode())

crypto{fr46m3n73d_b4ckup_vuln?}
